## Imports

# Files Upload

## Data Cleanup and analysis

Drop collumns that have null values or that have all the same value

In [44]:
drop_columns_with_all_nan(df_teams)
print(df_teams.duplicated().sum())
df_teams.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_teams_post)
print(df_teams_post.duplicated().sum())
df_teams_post.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_series_post)
print(df_series_post.duplicated().sum())
df_series_post.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_players)
print(df_players.duplicated().sum())
df_players.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_players_teams)
print(df_players_teams.duplicated().sum())
df_players_teams.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_coaches)
print(df_coaches.duplicated().sum())
print(df_coaches.duplicated(subset=['tmID', 'year']).sum())
df_coaches.drop_duplicates(inplace=True)

drop_columns_with_all_nan(df_awards_players)
print(df_awards_players.duplicated().sum())
df_awards_players.drop_duplicates(inplace=True)



[]
0
[]
0
[]
0
[]
0
[]
0
[]
0
20
[]
0


As we have the information about the year at the end of the playoff in each dataset, to avoid data leakage we will aggregate the data from the two previous years to the current year. And create new datasets by doing so.

### Teams

As there are new teams every year we decided to drop all the collumns as only the teams that played before had any relevant information. So as to not have any bias towards the teams that played before, all collumns were dropped.

### Teams post

In [45]:
df_teams_post_copy = df_teams_post.copy()
sum_collumns = ["W", "L"]
for year in years:
    for team in teams:
        for column in sum_collumns:
            df_teams_post_copy.loc[(df_teams_post_copy['year'] == year) & (df_teams_post_copy['tmID'] == team), column] = df_teams_post[(df_teams_post['year'] >= (year - years_back)) &(df_teams_post['year'] < year) & (df_teams_post['tmID'] == team)][column].mean()

df_teams_post_copy.fillna(0, inplace=True)
df_teams_post_copy.to_csv('data/teams_post_processed.csv', index=False)   


For each team some the number of win and losses they had in the year. For each year we then agregate the information from the previous two years.

In [46]:
winning_team_stats = df_series_post.groupby(["year", "tmIDWinner"])[["W"]].sum().reset_index()
winning_team_stats.columns = ["year", "tmID", "total_wins"]

losing_team_stats = df_series_post.groupby(["year", "tmIDLoser"])[["L"]].sum().reset_index()
losing_team_stats.columns = ["year", "tmID", "total_losses"]

# Merge the winning and losing team statistics
team_stats = winning_team_stats.merge(losing_team_stats, on=["year", "tmID"], how="outer").fillna(0)
team_stats_copy = team_stats.copy()
average_collumns = ["total_wins", "total_losses"]
for year in years:
    for team in teams:
        for column in average_collumns:
            team_stats_copy.loc[(team_stats_copy['year'] == year) & (team_stats_copy['tmID'] == team), column] = team_stats[(team_stats['year'] >= (year - years_back)) &(team_stats['year'] < year) & (team_stats['tmID'] == team)][column].mean()
team_stats_copy.fillna(0, inplace=True)
team_stats_copy.to_csv('data/series_post_processed.csv', index=False)

In [47]:
df_players_teams_copy = df_players_teams.copy()

average_collumns = ["GP","GS","minutes","points","oRebounds","dRebounds","rebounds","assists","steals","blocks","turnovers","PF","fgAttempted","fgMade","ftAttempted","ftMade","threeAttempted","threeMade","dq","PostGP","PostGS","PostMinutes","PostPoints","PostoRebounds","PostdRebounds","PostRebounds","PostAssists","PostSteals","PostBlocks","PostTurnovers","PostPF","PostfgAttempted","PostfgMade","PostftAttempted","PostftMade","PostthreeAttempted","PostthreeMade","PostDQ"]
for year in years:
    for team in teams:
        players = df_players_teams[(df_players_teams['year'] == year) & (df_players_teams['tmID'] == team)]['playerID'].unique()
        for player in players:
            for column in average_collumns:
                #df_players_teams_copy.loc[(df_players_teams_copy['year'] == year) & (df_players_teams_copy['tmID'] == team) & (df_players_teams_copy['playerID'] == player), column] = df_players_teams[(df_players_teams['year'] < year) & (df_players_teams['tmID'] == team) & (df_players_teams['playerID'] == player)][column].mean()
                df_players_teams_copy.loc[(df_players_teams_copy['year'] == year) & (df_players_teams_copy['tmID'] == team) & (df_players_teams_copy['playerID'] == player), column] = df_players_teams[(df_players_teams['year'] >= (year - years_back)) &(df_players_teams['year'] < year)  & (df_players_teams['playerID'] == player)][column].mean()

df_players_teams_copy.dropna(inplace=True)
df_players_teams_copy.drop(columns=['stint'], inplace=True)
df_players_teams_copy.to_csv('data/players_teams_processed.csv', index=False)

print(df_players_teams.info(), df_players_teams_copy.info())

KeyboardInterrupt: 

In [ ]:
df_coaches_copy = df_coaches.copy()

average_collumns = ["won","lost","post_wins","post_losses"]
for year in years:
    for team in teams:
        coaches= df_coaches[(df_coaches['year'] == year) & (df_coaches['tmID'] == team)]['coachID'].unique()
        for coach in coaches:
            for column in average_collumns:
                df_coaches_copy.loc[(df_coaches_copy['year'] == year) & (df_coaches_copy['tmID'] == team) & (df_coaches_copy['coachID'] == coach), column] = df_coaches[(df_coaches['year'] >= (year - years_back)) &(df_coaches['year'] < year) & (df_coaches['coachID'] == coach)][column].mean()         

df_coaches_copy.fillna(0, inplace=True)
df_coaches_copy.to_csv('data/coaches_processed.csv', index=False)


### Players

In [ ]:
def changeOutliersMean(data, column):
    #change the outliers with the mean
    #SFinding the IQR
    percentile25 = data[column].quantile(0.25)
    percentile75 = data[column].quantile(0.75)
    #Finding the upper and lower limits
    iqr = percentile75 - percentile25
    upper_limit = percentile75 + 1.5 * iqr
    lower_limit = percentile25 - 1.5 * iqr

    # Find outliers
    outliers = data[(data[column] > upper_limit) | (data[column] < lower_limit)]

    # Replace outliers with the mean of the 'height' column
    mean_height = data[column].mean()
    data.loc[outliers.index, column] = mean_height

Drop players from whom we have no information

In [ ]:
df_players = df_players[~((df_players['pos'].isna()) &
                            (df_players['height'] == 0.0) &
                            (df_players['weight'] == 0) &
                            (df_players['college'].isna()) &
                            (df_players['collegeOther'].isna()) &
                            (df_players['birthDate'] == "0000-00-00") &
                            (df_players['deathDate'] == "0000-00-00"))]
changeOutliersMean(df_players, 'height')
changeOutliersMean(df_players, 'weight')
df_players.to_csv('data/players_processed.csv', index=False)

### Awards

In [ ]:
award_counts = df_awards_players.groupby(["year", "playerID"]).size().reset_index(name="award_count")

# Create an empty DataFrame to store the results
award_counts_copy = pd.DataFrame(columns=["year", "playerID", "award_count"])
players = df_awards_players['playerID'].unique()
# Iterate over all possible combinations of "year" and "player" and add them to the DataFrame
for year in years:
    for player in players:
        award_count = award_counts[(award_counts['year'] >= (year - years_back)) & (award_counts['year'] < year) & (award_counts['playerID'] == player)]['award_count'].sum()
        new_row = {"year": year, "playerID": player, "award_count": award_count}
        award_counts_copy = pd.concat([award_counts_copy, pd.DataFrame([new_row])], ignore_index=True)

award_counts_copy.dropna(inplace=True)
award_counts_copy.to_csv('data/awards_players_processed.csv', index=False)